# Diferentes tipos de rendimiento

In [1]:
# Bajamos los ETF de yahoo
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

tickers = ['VOO', 'NAFTRAC.MX']
start_date = '2015-01-01'
end_date = '2021-01-01'
St = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

[*********************100%%**********************]  1 of 2 completed


2 Failed downloads:



[*********************100%%**********************]  1 of 2 completed

['NAFTRAC.MX', 'VOO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2015-01-01 -> 2021-01-01)')


In [2]:
# Obten rendimientos
rt = St.pct_change()

In [3]:
# Rendiiento aritmetico VOO
#Cuanto crece tu inversion expresado en dinero 
(1 + rt.VOO).prod()

1.0

In [4]:
# Rendimiento geometric
rg = ((1 +rt.VOO).prod()) ** (1/len(rt)) - 1 

ZeroDivisionError: division by zero

Precio * (1 + r_g) * N = PrecioFinal

In [ ]:
rg

In [ ]:
St.VOO.head(1).values[0] *  ( 1 + rg) ** (len(rt))

In [ ]:
# ¿Cuanto da por año el S&P?
rg * 252 

In [ ]:
160 * (1 + 0.1201) ** 6 

In [ ]:
# Valores esperados acciones
ve_usa  = rt.VOO.mean()
ve_mx = rt['NAFTRAC.MX'].mean()

# Volatilidad acciones 
s_usa = rt.VOO.std()
s_mx = rt['NAFTRAC.MX'].std()

# Correlacion
corr_usa_mx =  rt.corr()['NAFTRAC.MX']['VOO']

# Covarianza
cov_usa_mx =  rt.cov()['NAFTRAC.MX']['VOO']


$$\sigma^2_p = w_a^2 \sigma^2_a + w_b^2 \sigma_{b}^2 + 2 w_a w_b cov(a,b)$$


$$cov(a,b) = \sigma_a \sigma_b corr(a,b) $$

$$\sigma^2_p = w_a^2 \sigma^2_a + w_b^2 \sigma_{b}^2 + 2 w_a w_b \sigma_a \sigma_b corr(a,b)$$


In [ ]:
# Crea data frame con distintas correlaciones
def volatility_correlation(df, s_usa, s_mx, correlation):
    w1 = df['w_usa']
    w2 = df['w_mx']
    sP = np.sqrt(w1**2 * s_usa**2 + w2**2 * s_mx**2 + 2*correlation*s_usa * s_mx * w1*w2)
    return sP

w = np.linspace(0, 1, 100) 
df = pd.DataFrame({
    'w_usa': w, 
    'w_mx' : 1 - w, 
    'valor_esperado' : (w*ve_usa + (1 - w) * ve_mx) * 252

})

# x es el dataframe en todo el row 

df['volatility'] = df.apply(lambda x: volatility_correlation(x, s_usa, s_mx, corr_usa_mx), axis = 1)* np.sqrt(252) 

df['volatility_corr-1'] = df.apply(lambda x: volatility_correlation(x, s_usa, s_mx, -1), axis = 1) * np.sqrt(252) 

df['volatility_corr1'] = df.apply(lambda x: volatility_correlation(x, s_usa, s_mx,  1), axis = 1) * np.sqrt(252) 

df['volatility_corr0'] = df.apply(lambda x: volatility_correlation(x, s_usa, s_mx, 0), axis = 1) * np.sqrt(252) 

In [ ]:
# Como se ve con la real?
plt.plot(df.volatility, 

# Cra distintas correlaciones

In [ ]:
# Grafica todo junto 


## Varios activos

In [ ]:
tickers = ['VOO', 'VEA', 'VWO', 'VNQ', 'NAFTRAC.MX']
start_date = '2015-01-01'
end_date = '2021-01-01'
St = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
rt = St.pct_change()

In [ ]:
# Crea una función que retorne el rendimiento esperado y la varianza de un portafolio
def random_portafolio(returns):
    def rand_weights(n):
        k = np.random.rand(n)
        return k / sum(k)
    # Definimos las constantes
    means = rt.mean()
    sigma = returns.cov()
    # Obtenemos pesos
    n_act = len(rt.keys())
    w = rand_weights(n_act)
    # Rendimiento esperado
    re = w.dot(means) * 252
    # Varianza
    variance = w.dot(sigma).dot(w.T) 
    vol = np.sqrt(variance) * np.sqrt(252)
    return re, vol, w                  

random_portafolio(rt)

In [ ]:
# Simula 500 portafolios 
valores_esperados = []
volatilidades = [] 
pesos = [] 
for i in range(500):
    vel1, vol1, w1 = random_portafolio

    valores_esperados.append(vel1)
    volatilidades.append(vol1) 
    pesos.append(w1) 

df_final = pd.DataFrame({
    'valor_esperado' : valores_esperados, 
    'volatilidad' : volatilidades,
    'pesos' : pesos

})

In [ ]:
# Grafica


# ¿en que invertir para obtener el de mínima varianza?

# Más riesgo, más volatilidad?

In [ ]:
df_regresion = pd.DataFrame()
df_regresion['median'] = rt.mean() * 252 
df['volatilidades'] = rt.std() * np.sqrt(252)
plt.scatter(df_regresion.volatildades, df_regresion.medias)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = df_regresion[['volatilidades']]
y = df_regresion[['medias']]
model = LinearRegression().fit(X, y) 

In [ ]:
df_regresion['predicciones'] = model.predict(X)

In [ ]:
# Escriber
